# Import des outils / jeu de données

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
)

In [ ]:
np.random.seed(0)
sns.set_theme()

In [ ]:
df = pd.read_csv(
    "data/data-cleaned-feature-engineering.csv",
    sep=",",
    index_col="ID",
    parse_dates=True,
)

In [ ]:
df_transforme = pd.read_csv(
    "data/data-transformed.csv",
    sep=",",
    index_col="ID",
    parse_dates=True,
)

## Variables globales

In [ ]:
var_numeriques = [
    "Year_Birth",
    "Income",
    "Recency",
    "MntWines",
    "MntFruits",
    "MntMeatProducts",
    "MntFishProducts",
    "MntSweetProducts",
    "MntGoldProds",
    "NumDealsPurchases",
    "NumWebPurchases",
    "NumCatalogPurchases",
    "NumStorePurchases",
    "NumWebVisitsMonth",
]

In [ ]:
var_categoriques = [
    "Education",
    "Marital_Status",
    "Kidhome",
    "Teenhome",
    "AcceptedCmp1",
    "AcceptedCmp2",
    "AcceptedCmp3",
    "AcceptedCmp4",
    "AcceptedCmp5",
    "Response",
]

In [ ]:
LABELS = (0, 1)

In [ ]:
X = pd.get_dummies(df_transforme.drop(columns=["Response", "Dt_Customer"]))
y = df[["Response"]].astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Fonctions et variables utiles

In [ ]:
score_modeles = []

In [ ]:
def ajout_score(model, nom_modele, y_test, y_pred):
    """Ajoute le score F1 de la classe 1 à score_modeles."""
    clf_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).T

    score_f1_classe0 = clf_report.iloc[0, 2]
    score_f1_classe1 = clf_report.iloc[1, 2]

    score_modeles.extend(
        (
            [nom_modele, "score_f1_classe0", score_f1_classe0],
            [nom_modele, "score_f1_classe1", score_f1_classe1],
        )
    )

# Régression logistique

## Régression logistique simple

In [ ]:
sns.boxplot(df, x="Response", y="Recency")

In [ ]:
X = df_transforme[["Recency"]]

In [ ]:
y = df[["Response"]].astype(int)

In [ ]:
log_reg = LogisticRegression(random_state=0)

In [ ]:
log_reg.fit(X_train, y_train)

In [ ]:
y_pred = log_reg.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=log_reg.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=log_reg.classes_)
disp.plot()

In [ ]:
print(classification_report(y_test, y_pred, labels=log_reg.classes_))

In [ ]:
sns.regplot(
    data=df, x="Income", y="Response", logistic=True, ci=None, line_kws={"color": "red"}
)

## Régression logistique multiple

In [ ]:
X = pd.get_dummies(df_transforme.drop(columns=["Response"]))

In [ ]:
y = df[["Response"]].astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
model = LogisticRegression(random_state=0)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, labels=model.classes_))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot()

In [ ]:
nom_modele = "Régression logistique"
ajout_score(model, nom_modele, y_test, y_pred)

# Linear Discriminant Analysis

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Arbre de décision

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Support Vector Classifier

In [ ]:
from sklearn.svm import LinearSVC

# Naive Bayes

## Bernoulli Naive Bayes

In [ ]:
from sklearn.naive_bayes import BernoulliNB

## Complement Naive Bayes

In [ ]:
from sklearn.naive_bayes import ComplementNB

# k-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# XGBoost

In [ ]:
import xgboost

## Optimisation des hyper-paramètres

In [ ]:
params = {
    "max_depth": [3, 6, 10],
    "learning_rate": [0.01, 0.05, 0.1],
    "n_estimators": [100, 500, 1000],
    "colsample_bytree": [0.3, 0.7],
}

In [ ]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(
    estimator=model,
    param_grid=params,
    scoring="precision",
    verbose=1,
)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
model = clf

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, labels=model.classes_))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot()

In [ ]:
nom_modele = "XGBoost (GridSearchCV)"
ajout_score(model, nom_modele, y_test, y_pred)

# Réseau de neurones

In [ ]:
X_train = np.asarray(X_train).astype("float32")
y_train = np.asarray(y_train).astype("float32")
X_test = np.asarray(X_test).astype("float32")
y_test = np.asarray(y_test).astype("float32")

In [ ]:
X_train.shape

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

np.random.seed(0)
model = keras.Sequential(
    [
        layers.Dense(400, activation="relu", input_shape=[X_train.shape[1]]),
        layers.Dense(400, activation="relu"),
        layers.Dense(400, activation="sigmoid"),
        layers.Dense(400, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

In [ ]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["binary_accuracy"],
)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    # validation_split=0.2,
    batch_size=512,
    epochs=1000,
    callbacks=[early_stopping],
    verbose=0,  # hide the output because we have so many epochs
)

In [ ]:
history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5
history_df.loc[5:, ["loss", "val_loss"]].plot()
history_df.loc[5:, ["binary_accuracy", "val_binary_accuracy"]].plot()

print(
    ("Best Validation Loss: {:0.4f}" + "\nBest Validation Accuracy: {:0.4f}").format(
        history_df["val_loss"].min(), history_df["val_binary_accuracy"].max()
    )
)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
sns.histplot(y_pred > 0.5, kde=True)

In [ ]:
y_pred_old = y_pred

In [ ]:
y_pred = y_pred > 0.5

In [ ]:
print(classification_report(y_test, y_pred))  # , labels=model.classes_))

In [ ]:
cm = confusion_matrix(y_test, y_pred)  # , labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)  # , display_labels=model.classes_)
disp.plot()

In [ ]:
nom_modele = "Réseau de Neurones"
ajout_score(model, nom_modele, y_test, y_pred)

# Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
model = VotingClassifier(
    estimators=[
        ("lr", LogisticRegression(random_state=0)),
        ("dt", DecisionTreeClassifier(random_state=0)),
        ("lda", LinearDiscriminantAnalysis()),
        ("xgb", xgboost.XGBClassifier(random_state=0)),
    ],
    voting="soft",
)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, labels=model.classes_))

In [ ]:
clf_report = pd.DataFrame(
    classification_report(y_test, y_pred, labels=model.classes_, output_dict=True)
).T

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot()

In [ ]:
nom_modele = "Voting Classifier"
ajout_score(model, nom_modele, y_test, y_pred)

# Équilibrage des classes

In [ ]:
samples0 = df_transforme[df_transforme["Response"] == 0].sample(350, random_state=0)

In [ ]:
Xnew = pd.concat((samples0, df_transforme[df_transforme["Response"] == 1]))

In [ ]:
print(Xnew)

In [ ]:
Xnew["Response"].hist()

In [ ]:
X = pd.get_dummies(Xnew.drop(columns=["Response", "Dt_Customer"]))
y = Xnew[["Response"]].astype(int)

In [ ]:
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## XGBoost

In [ ]:
model = xgboost.XGBClassifier(random_state=0)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, labels=model.classes_))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=model.classes_, normalize="true")
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot()

## Neural Network

In [ ]:
X_train = np.asarray(X_train).astype("float32")
y_train = np.asarray(y_train).astype("float32")
X_test = np.asarray(X_test).astype("float32")
y_test = np.asarray(y_test).astype("float32")

In [ ]:
X_train.shape

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

np.random.seed(0)
model = keras.Sequential(
    [
        layers.Dense(400, activation="relu", input_shape=[X_train.shape[1]]),
        layers.Dense(400, activation="relu"),
        layers.Dense(400, activation="sigmoid"),
        layers.Dense(400, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

In [ ]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["binary_accuracy"],
)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    # validation_split=0.2,
    batch_size=512,
    epochs=1000,
    callbacks=[early_stopping],
    verbose=0,  # hide the output because we have so many epochs
)

In [ ]:
history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5
history_df.loc[5:, ["loss", "val_loss"]].plot()
history_df.loc[5:, ["binary_accuracy", "val_binary_accuracy"]].plot()

print(
    ("Best Validation Loss: {:0.4f}" + "\nBest Validation Accuracy: {:0.4f}").format(
        history_df["val_loss"].min(), history_df["val_binary_accuracy"].max()
    )
)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
sns.histplot(y_pred > 0.5, kde=True)

In [ ]:
y_pred_old = y_pred

In [ ]:
y_pred = y_pred > 0.5

In [ ]:
print(classification_report(y_test, y_pred))  # , labels=model.classes_))

In [ ]:
cm = confusion_matrix(y_test, y_pred, normalize="true")
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

## Truc Gaussien

In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier

In [ ]:
model = GaussianProcessClassifier(random_state=0)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, labels=model.classes_))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=model.classes_, normalize="true")

In [ ]:
sns.heatmap(cm, annot=True, cmap="Purples", vmin=0, vmax=1)

# Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
iforest = IsolationForest(random_state=0)

In [ ]:
iforest.fit(X)

In [ ]:
X.head()

In [ ]:
sns.histplot(iforest.predict(X))

In [ ]:
X["outlier"] = iforest.predict(X)

In [ ]:
X

In [ ]:
sns.histplot(data=X, hue="outlier", x="Income", bins=30)

# Refactor

In [ ]:
models = [
    [LogisticRegression(random_state=0), "LogisticRegression"],
    [LinearDiscriminantAnalysis(), "LinearDiscriminantAnalysis"],
    [DecisionTreeClassifier(random_state=0), "DecisionTreeClassifier"],
    [RandomForestClassifier(random_state=0), "RandomForestClassifier"],
    [LinearSVC(random_state=0), "LinearSVC"],
    [BernoulliNB(), "BernoulliNB"],
    [ComplementNB(), "ComplementNB"],
    [KNeighborsClassifier(), "KNeighborsClassifier"],
    [xgboost.XGBClassifier(random_state=0), "XGBClassifier"],
]

In [ ]:
print(type(model.classes_))

In [ ]:
results = []

for model, model_name in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    cm = confusion_matrix(y_test, y_pred, labels=LABELS)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=LABELS)
    # sns.heatmap(cm, annot=True, cmap="Purples", vmin=0, vmax=1)

    clf_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).T

    score_f1_classe0 = clf_report.iloc[0, 2]
    score_f1_classe1 = clf_report.iloc[1, 2]

    results.append([model_name, score_f1_classe1])
    # print(f"{model_name} : {score_f1_classe1}")
    # disp.plot()

In [ ]:
results.sort(key=lambda x: x[1], reverse=True)
results

# Sauvegarde des données

In [ ]:
score_modeles_df = pd.DataFrame(score_modeles, columns=["Modèle", "Métrique", "Valeur"])

In [ ]:
score_modeles_df.to_csv("data/results/classifications.csv")